# TP 2 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
DISPLAY_TREE = true
if DISPLAY_TREE
    # Les 2 lignes suivantes à décommenter sur les machines persos
    import Pkg; 
    Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
    using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound
end

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
           ✗ GR
           ✗ Plots
           ✗ Plots → UnitfulExt
           ✗ Plots → IJuliaExt
           ✗ Plots → GeometryBasicsExt
  0 dependencies successfully precompiled in 35 seconds. 235 already precompiled.
  5 dependencies errored.
  For a report of the errors see `julia> err`. To retry use `pkg> precompile`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
           ✗ GR
           ✗ Plots
           ✗ Plots → IJuliaExt
           ✗ Plots → GeometryBasicsExt
           ✗ Plots → UnitfulExt
  0 dependencies successfully precompiled in 34 seconds. 235 already precompiled.
  5 dependencies errored.
  For a report of the errors see `julia> err`. To retry use `pkg> precompile`
[ 

LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to "/home/cycy/.julia/compiled/v1.11/Plots/jl_0eWj1b".

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [3]:
function testSondability_LP(model2, x, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(x)]-value.(x)) .<= fill(10^-5, size(x)))
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (objective_value(model2) >= BestProfit)
            Bestsol = value.(x)
            #BestProfit=value(benef)
            BestProfit=objective_value(model2)
            println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

testSondability_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [4]:
function separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left

    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end

    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible

    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop
end

exploreNextNode_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [5]:
# Les 3 lignes suivantes à décommenter sur les machines persos
import Pkg; 
Pkg.add("Clp");
Pkg.add("JuMP");
using JuMP, Clp #to comment for the lab class

function updateModele_LP!(model2, x, listobjs, listvals)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],listvals[i])
        set_upper_bound(x[listobjs[i]],listvals[i])
    end
end

function reset_LP!(model2, x, listobjs)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],0.0)
        set_upper_bound(x[listobjs[i]],1.0)
    end
end

function resetAll_LP!(model2, x)
    for i in 1:length(x)
        set_lower_bound(x[i],0.0)
        set_upper_bound(x[i],1.0)
    end
end

function createModel_LP(price, weight, capacity)
# ROOT NODE

    n=length(price)

    model2 = Model(Clp.Optimizer) # set optimizer
    set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
    set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

    # define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
    @variable(model2, 0 <= x[i in 1:n] <= 1)

    # define objective function
    @objective(model2, Max, sum(price[i]*x[i] for i in 1:n))

    # define the capacity constraint
    @constraint(model2, sum(weight[i]*x[i] for i in 1:n) <=  capacity)

    println(model2)

    return model2, x
end

   Resolving package versions...
   Installed CodecBzip2 ─────────── v0.8.4
   Installed MUMPS_seq_jll ──────── v500.700.300+0
   Installed CoinUtils_jll ──────── v200.1100.1100+0
   Installed OpenBLAS32_jll ─────── v0.3.28+3
   Installed BenchmarkTools ─────── v1.5.0
   Installed MutableArithmetics ─── v1.6.0
   Installed SpecialFunctions ───── v2.5.0
   Installed METIS_jll ──────────── v5.1.2+1
   Installed DiffRules ──────────── v1.15.1
   Installed CommonSubexpressions ─ v0.3.1
   Installed MathOptInterface ───── v1.34.0
   Installed Osi_jll ────────────── v0.10800.1000+0
   Installed DiffResults ────────── v1.1.0
   Installed ZMQ ────────────────── v1.4.0
   Installed DataStructures ─────── v0.18.20
   Installed Clp_jll ────────────── v100.1700.901+0
   Installed OpenSpecFun_jll ────── v0.5.5+0
   Installed Clp ────────────────── v1.2.1
   Installed ForwardDiff ────────── v0.10.38
    Updating `~/.julia/environments/v1.11/Project.toml`
  [e2554f3b] + Clp v1.2.1
    Updating `~/.ju

createModel_LP (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [6]:
function solveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)

    model2, x = createModel_LP(price, weight, capacity)
    
    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree
    
    #intermediate structure to navigate in the search tree
    listobjs=Int64[]
    listvals=Float64[]
    listnodes=Int64[]

    BestProfit::Float64=-1.0
    Bestsol=Float64[]

    current_node_number::Int64=0
    stop = false

    while(!stop)

        println("\nNode number ", current_node_number, ": \n---------------\n")

        if DISPLAY_TREE
        #Update the graphical tree
            push!(trNamenodes,current_node_number+1) 
            if(length(trNamenodes)>=2)
                push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
                push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
            end
        end
        
        push!(listnodes, current_node_number)
        
        #create LP of current node
        updateModele_LP!(model2, x, listobjs, listvals)
        
        println(model2)
        
        print("Solve the LP model of the current node to compute its bound: start ... ")

        status = optimize!(model2)

        println("... end"); 

        print(": Solution LP")
        if(termination_status(model2) == MOI.INFEASIBLE)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", objective_value(model2))
            [print("\t", name(v),"=",value(v)) for v in all_variables(model2)] 
        end
        println(" "); 


        println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")

        TA, TO, TR, Bestsol, BestProfit = testSondability_LP(model2, x, BestProfit, Bestsol)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, length(price))
        else
            stop = exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
        end
        
        resetAll_LP!(model2, x)

        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


solveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [7]:
function solveNdisplayKnap(filename)

    println("\n Branch-and-Bound for solving a knapsack problem. \n\n Solving instance '" * filename * "'\n")

    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = solveKnapInstance(filename)

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
    if DISPLAY_TREE
        println("\n Branch-and-bound tree visualization : start display ...")
        display(graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree))
        println("... end display. \n\n")
    end
end

solveNdisplayKnap (generic function with 1 method)

In [8]:
INSTANCE = "InstancesKnapSack/test.opb.txt"

solveNdisplayKnap(INSTANCE)


 Branch-and-Bound for solving a knapsack problem. 

 Solving instance 'InstancesKnapSack/test.opb.txt'

Max 42 x[1] + 40 x[2] + 12 x[3] + 25 x[4]
Subject to
 7 x[1] + 4 x[2] + 3 x[3] + 5 x[4] ≤ 10
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[1] ≤ 1
 x[2] ≤ 1
 x[3] ≤ 1
 x[4] ≤ 1


Node number 0: 
---------------

Max 42 x[1] + 40 x[2] + 12 x[3] + 25 x[4]
Subject to
 7 x[1] + 4 x[2] + 3 x[3] + 5 x[4] ≤ 10
 x[1] ≥ 0
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[1] ≤ 1
 x[2] ≤ 1
 x[3] ≤ 1
 x[4] ≤ 1

Solve the LP model of the current node to compute its bound: start ... ... end
: Solution LP 76.0	x[1]=0.857142857142857	x[2]=1.0	x[3]=0.0	x[4]=0.0 

Previous Solution memorized Float64[] with bestprofit -1.0

non sondable

branch on object 1


Node number 1: 
---------------

Max 42 x[1] + 40 x[2] + 12 x[3] + 25 x[4]
Subject to
 7 x[1] + 4 x[2] + 3 x[3] + 5 x[4] ≤ 10
 x[1] ≥ 1
 x[2] ≥ 0
 x[3] ≥ 0
 x[4] ≥ 0
 x[1] ≤ 1
 x[2] ≤ 1
 x[3] ≤ 1
 x[4] ≤ 1

Solve the LP model of the current node to compute its bound: 

LoadError: MethodError: no method matching plot(::GraphRecipes.GraphPlot; names::Vector{Any}, method::Symbol)
The function `plot` exists, but no method is defined for this combination of argument types.

### **QUESTION PRELIMINAIRE 1 :**
Dans l'exemple, on sépare les noeuds dans l'ordre lexicographique des variables qui n'ont pas encore été fixées. C'est-à-dire qu'au premier noeud on sépare sur X1 puis en profondeur 2 de l'arbre ont sépare sur la variable X2 et ainsi de suite.

### **QUESTION PRELIMINAIRE 2 :**

### **QUESTION PRELIMINAIRE 3 :**

### **QUESTION PRELIMINAIRE 4:**


In [ ]:
function best_ratio_in_not_fixed_objects(price, weight, listobjs, listvals)
    # à finir calcul du best ratio parmis les objets qui ne sont pas dans listobjs
    # ratios = price / weight;
    # return sort!(ratio)
end

function borne1(price, weight, capacity, listobjs, listvals)

    n = length(listobjs);
    
    price, weight, capacity = readKnaptxtInstance("test.opb.txt");

    bornesup = 0;
    remaining_capacity = capacity;
    for i in 1:n
        remaining_capacity = remaining_capacity - listvals[i] * weight[listobjs[i]];
        bornesup += listvals[i] * price[listobjs[i]];
    end

    best_ratio = best_ratio_in_not_fixed_objects(price, weight, listobjs, listvals);
    bornesup = capacity * best_ratio;

    # à finir : maj listobjs et listvals
    
    return bornesup, listobjs, listvals
end